In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1598095688600_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [2]:
from pyspark.sql import functions as func
#Specifying the schema instead of inferring it 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType

fileSchema = StructType([StructField('movie_id', IntegerType(),True),
                        StructField('user_id', IntegerType(),True),
                        StructField('rating', IntegerType(),True),
                        StructField('time_stamp', LongType(),True)])

movie_ratings = spark.read.load("s3://sparkbucket123/u.data", format="csv", sep="\t", schema = fileSchema, header="true")
movie_ratings.show();

movie_rating_count = movie_ratings.groupBy('movie_id').agg(func.count("rating"))

movie_rating_count.show()


#Reading the movie id and movie name
movies = spark.read.load("s3://sparkbucket123/u.item", format="csv", sep="|", inferSchema="true", header="true")
movie_selected = movies.select('10', 'Toy Story (1995)')
from functools import reduce
oldColumns = movie_selected.schema.names
newColumns = ["movie_id", "movie_name"]
movie_names = reduce(lambda movie_selected, idx: movie_selected.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), movie_selected)
movie_names.show()


#Joining the two movies data
joined_movie_data = movie_ratings.join(movie_names , ["movie_id"])
joined_movie_data.show()


#Finding which movie name got how many ratings
movie_name_ratings_count = joined_movie_data.groupBy('movie_name').agg(func.count("rating"))
movie_name_ratings_count.show()


#Saving this data in the disk to be used later
movie_name_ratings_count.write.json("movie_ratings.json")






VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------+----------+
|movie_id|user_id|rating|time_stamp|
+--------+-------+------+----------+
|     186|    302|     3| 891717742|
|      22|    377|     1| 878887116|
|     244|     51|     2| 880606923|
|     166|    346|     1| 886397596|
|     298|    474|     4| 884182806|
|     115|    265|     2| 881171488|
|     253|    465|     5| 891628467|
|     305|    451|     3| 886324817|
|       6|     86|     3| 883603013|
|      62|    257|     2| 879372434|
|     286|   1014|     5| 879781125|
|     200|    222|     5| 876042340|
|     210|     40|     3| 891035994|
|     224|     29|     3| 888104457|
|     303|    785|     3| 879485318|
|     122|    387|     5| 879270459|
|     194|    274|     2| 879539794|
|     291|   1042|     4| 874834944|
|     234|   1184|     2| 892079237|
|     119|    392|     4| 886176814|
+--------+-------+------+----------+
only showing top 20 rows

+--------+-------------+
|movie_id|count(rating)|
+--------+-------------+
|     148| 

In [3]:
#Reading it later
movie_ratings_stored=spark.read.json("movie_ratings.json")
movie_ratings_stored.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+
|count(rating)|          movie_name|
+-------------+--------------------+
|           89| Dante's Peak (1997)|
|           44|   Carrington (1995)|
|          150|    Liar Liar (1997)|
|           27|      Air Bud (1997)|
|           25|  Deep Rising (1998)|
|          238|Smilla's Sense of...|
|          283|Conspiracy Theory...|
|          357|Mary Shelley's Fr...|
|          166| Paris, Texas (1984)|
|          183|Kiss the Girls (1...|
|           63|       Aliens (1986)|
|           43|My Life as a Dog ...|
|          155|Much Ado About No...|
|           23| Bed of Roses (1996)|
|          124|    Mr. Magoo (1997)|
|          124|     Rosewood (1997)|
|           21|Wings of Desire (...|
|           49|Bloody Child, The...|
|           46|Hunt for Red Octo...|
|          187|Grand Day Out, A ...|
+-------------+--------------------+
only showing top 20 rows